In [3]:
import pyreadr
import pandas as pd

In [5]:
# Reading in Data
dataset = pyreadr.read_r("../data/brain.rda")

In [21]:
arraymeta = dataset['arraymeta']

,patient,region.ancg,region.cb,region.dlpfc,lab.davis,lab.irvine,lab.michigan,age,sex,arrayversion
1,1834,1.0,0.0,0.0,0.0,1.0,0.0,40,M,1.0
149,1834,0.0,1.0,0.0,0.0,1.0,0.0,40,M,1.0
191,1834,0.0,1.0,0.0,0.0,0.0,1.0,40,M,1.0
209,1834,0.0,0.0,1.0,1.0,0.0,0.0,40,M,1.0
210,1834,0.0,0.0,1.0,1.0,0.0,0.0,40,M,2.0
...,...,...,...,...,...,...,...,...,...,...
126,4236,1.0,0.0,0.0,0.0,0.0,1.0,63,M,2.0
385,4236,0.0,0.0,1.0,0.0,0.0,1.0,63,M,2.0
66,4250,1.0,0.0,0.0,0.0,1.0,0.0,64,M,2.0
127,4250,1.0,0.0,0.0,0.0,0.0,1.0,64,M,2.0


In [23]:
dataset['

ValueError: cannot convert an unnamed DataArray to a DataFrame: use the ``name`` parameter